In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing
import pandas as pd

# Datos no numéricos

In [ ]:
df = pd.read_excel('titanic.xls')

In [ ]:
original_df = pd.DataFrame.copy(df)

In [ ]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
df.drop(['body','name'], axis = 1, inplace=True) # Creemos que no importan

In [ ]:
df.fillna(0,inplace=True) # Para evitar nulls

In [ ]:
def handle_non_numerical_data(df):     # Crear categorías para datos no numéricos
  columns = df.columns.values

  for column in columns:
    text_digit_vals = {} # Creamos un diccionario vacío que servirá para asignar cada valor no numérico a numérico
    def convert_to_int(val):
      return text_digit_vals[val] # Le llega el valor, ejemplo {'Female':0} Lo convertirá a 0
    if df[column].dtype != np.int64 and df[column].dtype != np.float64: # Si no es un número
      column_contents=df[column].values.tolist() # Entonces convierte los valores de la columna a una lista.
      unique_elements=set(column_contents) # Solo valores diversos
      x = 0 # Para comenzar las etiquetas desde 0, es por eso que {'Female':0}
      for unique in unique_elements:
        if unique not in text_digit_vals:
          text_digit_vals[unique] = x # Se asigna el valor y luego aumenta más 1
          x+=1
      df[column]=list(map(convert_to_int,df[column])) # Se crea el diccionario
  return df

  # Estudiar más funciones de mapeo.

In [ ]:
df = handle_non_numerical_data(df)
df

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,0,29.0000,0,0,749,211.3375,36,2,2,189
1,1,1,1,0.9167,1,2,509,151.5500,61,2,11,155
2,1,0,0,2.0000,1,2,509,151.5500,61,2,0,155
3,1,0,1,30.0000,1,2,509,151.5500,61,2,0,155
4,1,0,0,25.0000,1,2,509,151.5500,61,2,0,155
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,0,14.5000,1,0,418,14.4542,0,3,0,0
1305,3,0,0,0.0000,1,0,418,14.4542,0,3,0,0
1306,3,0,1,26.5000,0,0,672,7.2250,0,3,0,0
1307,3,0,1,27.0000,0,0,522,7.2250,0,3,0,0


In [ ]:
df.drop(['ticket','home.dest'], axis = 1, inplace=True)

In [ ]:
X = np.array(df.drop(['survived'], axis = 1).astype(float)) # Preparando las features

In [ ]:
X = preprocessing.scale(X)

In [ ]:
y = np.array(df['survived']) # Preparando el outcome

In [ ]:
clf = MeanShift() # clf = classifier
clf.fit(X)

MeanShift()

In [ ]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

In [ ]:
original_df['cluster_group']=np.nan

In [ ]:
for i in range (len(X)):
  original_df['cluster_group'].iloc[i] = labels[i]

Se han truncado las últimas 5000 líneas del flujo de salida.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  original_df['cluster_group'].iloc[i] = labels[i]
/tmp/ipython-input-2488596260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_df['cluster_group'].iloc[i] = labels[i]
/tmp/ipython-input-2488596260.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as 

In [ ]:
n_clusters_ = len(np.unique(labels))

In [ ]:
survival_rates = {}
for i in range(n_clusters_):
  temp_df = original_df[(original_df['cluster_group']==float(i))]
  survival_cluster=temp_df[(temp_df['survived']==1)]
  survival_rate = len(survival_cluster)/len(temp_df)
  survival_rates[1] = survival_rate

In [ ]:
print(survival_rate) # 5:20 minuto

0.6060606060606061
